In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Psoriasis"
cohort = "GSE183134"

# Input paths
in_trait_dir = "../../input/GEO/Psoriasis"
in_cohort_dir = "../../input/GEO/Psoriasis/GSE183134"

# Output paths
out_data_file = "../../output/preprocess/Psoriasis/GSE183134.csv"
out_gene_data_file = "../../output/preprocess/Psoriasis/gene_data/GSE183134.csv"
out_clinical_data_file = "../../output/preprocess/Psoriasis/clinical_data/GSE183134.csv"
json_path = "../../output/preprocess/Psoriasis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptomic profiling of Pityriasis Rubra Pilaris (PRP) and Psoriasis"
!Series_summary	"The microarray experiment was employed to evaluate the gene expressions in skin lesions of PRP and psoriasis."
!Series_overall_design	"To investigate the specific gene regulations, microarray profiling was performed on RNA extracted from paraffin embedded skin biopsy samples."
Sample Characteristics Dictionary:
{0: ['tissue: Skin'], 1: ['disease state: Pityriasis_Rubra_Pilaris', 'disease state: Psoriasis']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this is a microarray profiling study,
# so it likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Checking the Sample Characteristics Dictionary
# The trait data (disease state) is available in row 1
trait_row = 1
# No age information is available
age_row = None
# No gender information is available
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait value to binary format (0 for PRP, 1 for Psoriasis)"""
    if value is None:
        return None
    # Extract the value part if it contains a colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary (1 for Psoriasis, 0 for PRP)
    if "psoriasis" in value.lower():
        return 1
    elif "pityriasis_rubra_pilaris" in value.lower() or "prp" in value.lower():
        return 0
    return None

def convert_age(value):
    """Convert age value to numeric format"""
    # Not needed as age data is not available
    return None

def convert_gender(value):
    """Convert gender value to binary format (0 for female, 1 for male)"""
    # Not needed as gender data is not available
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None
# Initial validation
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait data is available, extract clinical features
if trait_row is not None:
    # Assuming clinical_data is already defined from previous step
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save to CSV
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM5551681': [0.0], 'GSM5551682': [0.0], 'GSM5551683': [0.0], 'GSM5551684': [0.0], 'GSM5551685': [0.0], 'GSM5551686': [0.0], 'GSM5551687': [0.0], 'GSM5551688': [0.0], 'GSM5551689': [0.0], 'GSM5551690': [0.0], 'GSM5551691': [0.0], 'GSM5551692': [0.0], 'GSM5551693': [0.0], 'GSM5551694': [1.0], 'GSM5551695': [1.0], 'GSM5551696': [1.0], 'GSM5551697': [1.0], 'GSM5551698': [1.0], 'GSM5551699': [1.0], 'GSM5551700': [1.0], 'GSM5551701': [1.0], 'GSM5551702': [1.0], 'GSM5551703': [1.0], 'GSM5551704': [1.0], 'GSM5551705': [1.0], 'GSM5551706': [1.0], 'GSM5551707': [1.0], 'GSM5551708': [1.0], 'GSM5551709': [1.0], 'GSM5551710': [1.0], 'GSM5551711': [1.0], 'GSM5551712': [1.0], 'GSM5551713': [1.0], 'GSM5551714': [1.0], 'GSM5551715': [1.0]}
Clinical data saved to ../../output/preprocess/Psoriasis/clinical_data/GSE183134.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['1-Dec', '1-Sep', '10-Mar', '10-Sep', '11-Mar', '11-Sep', '12-Sep',
       '14-Sep', '15-Sep', '2-Sep', '3-Mar', '3-Sep', '4-Mar', '4-Sep',
       '5-Mar', '6-Mar', '6-Sep', '7-Mar', '7-Sep', '8-Mar'],
      dtype='object', name='ID')

Gene data dimensions: 29405 genes × 35 samples


### Step 4: Gene Identifier Review

In [5]:
# Examine the gene identifiers in the dataset
# The identifiers appear to be non-standard gene symbols (e.g., "1-Dec", "1-Sep", "10-Mar")
# These are likely probe identifiers or some other format that requires mapping to standard gene symbols

# Based on biomedical knowledge, standard human gene symbols would follow HGNC nomenclature
# Examples of standard gene symbols: BRCA1, TP53, TNF, IL6, etc.
# The identifiers seen here (like "1-Dec", "3-Mar") don't conform to standard gene symbol conventions

# These identifiers need to be mapped to standard gene symbols for proper analysis
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Inspect the SOFT file structure to understand the annotation format
# Read the first few lines of the SOFT file to examine its structure
import gzip
print("Preview of SOFT file content:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        print(line.strip())
        if i >= 20:  # Print first 20 lines to understand structure
            break
print("...\n")

# 3. Try different approaches to extract gene annotation data
# First, let's try the default method to see what's actually in the file
gene_annotation = get_gene_annotation(soft_file)
print("Gene annotation dataframe using default method:")
print(f"Shape: {gene_annotation.shape}")
print(f"Columns: {gene_annotation.columns.tolist()}")
print(gene_annotation.head(3))

# 4. Check if there's another section in the file that might contain the mapping
# Look for platform annotation information in the SOFT file
print("\nSearching for platform annotation section in SOFT file...")
with gzip.open(soft_file, 'rt') as f:
    platform_lines = []
    capture = False
    for i, line in enumerate(f):
        if "^PLATFORM" in line:
            capture = True
            platform_lines.append(line.strip())
        elif capture and line.startswith("!platform_table_begin"):
            platform_lines.append(line.strip())
            for j in range(10):  # Capture the next 10 lines to understand the table structure
                try:
                    platform_line = next(f).strip()
                    platform_lines.append(platform_line)
                except StopIteration:
                    break
            break
    
    print("\n".join(platform_lines))

# Maintain gene availability status as True based on previous steps
is_gene_available = True


Preview of SOFT file content:
^DATABASE = GeoMiame
!Database_name = Gene Expression Omnibus (GEO)
!Database_institute = NCBI NLM NIH
!Database_web_link = http://www.ncbi.nlm.nih.gov/geo
!Database_email = geo@ncbi.nlm.nih.gov
^SERIES = GSE183134
!Series_title = Transcriptomic profiling of Pityriasis Rubra Pilaris (PRP) and Psoriasis
!Series_geo_accession = GSE183134
!Series_status = Public on Sep 30 2021
!Series_submission_date = Aug 31 2021
!Series_last_update_date = Jan 17 2022
!Series_pubmed_id = 34491907
!Series_summary = The microarray experiment was employed to evaluate the gene expressions in skin lesions of PRP and psoriasis.
!Series_overall_design = To investigate the specific gene regulations, microarray profiling was performed on RNA extracted from paraffin embedded skin biopsy samples.
!Series_type = Expression profiling by array
!Series_contributor = Johann,E,Gudjonsson
!Series_contributor = Lam,C,Tsoi
!Series_sample_id = GSM5551681
!Series_sample_id = GSM5551682
!Series_sa

Gene annotation dataframe using default method:
Shape: (1058615, 2)
Columns: ['ID', 'SPOT_ID']
          ID    SPOT_ID
0    DDX11L1    DDX11L1
1  MIR1302-2  MIR1302-2
2      OR4F5      OR4F5

Searching for platform annotation section in SOFT file...
^PLATFORM = GPL30572
!platform_table_begin
ID	SPOT_ID
DDX11L1	DDX11L1
MIR1302-2	MIR1302-2
OR4F5	OR4F5
LOC100132287	LOC100132287
LOC105379690	LOC105379690
OR4F29	OR4F29
LOC105378947	LOC105378947
LOC105378580	LOC105378580
LOC100287934	LOC100287934


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Examine both gene identifiers and annotation to determine mapping columns
# From previous steps, we can see:
# - The gene expression data has identifiers like "1-Dec", "1-Sep", "10-Mar" as index
# - The gene annotation data shows columns "ID" and "SPOT_ID"

# Since the gene annotation DataFrame doesn't seem to contain our probe identifiers directly,
# we need to extract more detailed annotation from the SOFT file

# Let's specifically look for the platform annotation that contains our probe IDs
with gzip.open(soft_file, 'rt') as f:
    platform_lines = []
    capture = False
    for line in f:
        if "!platform_table_begin" in line:
            capture = True
            continue
        elif "!platform_table_end" in line:
            capture = False
            break
        elif capture:
            platform_lines.append(line.strip())

# Create a DataFrame from the platform lines if we found data
if platform_lines:
    import io
    platform_df = pd.read_csv(io.StringIO('\n'.join(platform_lines)), sep='\t')
    print(f"Platform annotation columns: {platform_df.columns.tolist()}")
    print(f"First few rows of platform annotation:")
    print(platform_df.head())
else:
    # If we couldn't find proper annotation, create a mapping from the gene expression data
    # and annotation we already have
    print("Could not find detailed probe-to-gene mapping in platform annotation.")
    # We'll proceed with the gene annotation we already extracted

# 2. Get mapping between gene identifiers and gene symbols
# Based on the output from previous steps, both ID and SPOT_ID columns in gene_annotation
# appear to contain gene symbols. Let's check if either matches our expression data index

# First, check if any IDs in gene_annotation match the expression data index
matching_ids = len(set(gene_data.index) & set(gene_annotation['ID']))
print(f"Number of matching IDs between expression data and gene_annotation['ID']: {matching_ids}")

# If we didn't find matches, the gene identifiers might be in a different format
# Let's try to extract mapping from additional annotation or metadata
if matching_ids == 0:
    print("No direct matches found. Creating alternative mapping...")
    
    # For this dataset, it appears that the gene identifiers are non-standard, 
    # but the annotation provides standard gene symbols
    # We'll create a simple 1:1 mapping using the annotation we have
    
    # Create a mapping dataframe where both probe ID and gene symbol are the same
    # Since our gene identifiers don't seem to map directly to standard annotations,
    # we'll use them as-is and treat them as approximations of gene symbols
    mapping_df = pd.DataFrame({
        'ID': gene_data.index,
        'Gene': gene_data.index
    })
    
    print("Created mapping with gene identifiers as gene symbols.")
    print(f"Mapping dataframe shape: {mapping_df.shape}")
    print(mapping_df.head())
else:
    # If we found matching IDs, use them for mapping
    mapping_df = gene_annotation[['ID', 'SPOT_ID']].rename(columns={'SPOT_ID': 'Gene'})
    mapping_df = mapping_df[mapping_df['ID'].isin(gene_data.index)]
    print(f"Mapping dataframe shape: {mapping_df.shape}")
    print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data_mapped = apply_gene_mapping(gene_data, mapping_df)
print(f"Mapped gene expression data shape: {gene_data_mapped.shape}")
print(f"First few gene symbols: {list(gene_data_mapped.index[:5])}")

# Update gene_data to use the mapped values
gene_data = gene_data_mapped

# Maintain gene availability status
is_gene_available = True


Platform annotation columns: ['ID', 'SPOT_ID']
First few rows of platform annotation:
             ID       SPOT_ID
0       DDX11L1       DDX11L1
1     MIR1302-2     MIR1302-2
2         OR4F5         OR4F5
3  LOC100132287  LOC100132287
4  LOC105379690  LOC105379690
Number of matching IDs between expression data and gene_annotation['ID']: 29405
Mapping dataframe shape: (1058580, 2)
             ID          Gene
0       DDX11L1       DDX11L1
1     MIR1302-2     MIR1302-2
2         OR4F5         OR4F5
3  LOC100132287  LOC100132287
4  LOC105379690  LOC105379690


Mapped gene expression data shape: (22785, 35)
First few gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Check if gene data is available after mapping
if gene_data.shape[0] == 0:
    print("Error: Gene expression matrix is empty after mapping.")
    # Mark the dataset as not usable due to lack of gene expression data
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=False,  # No usable gene data
        is_trait_available=True,
        is_biased=True,
        df=pd.DataFrame(),
        note="Failed to map probe IDs to gene symbols. The annotation format may not be compatible with the extraction methods."
    )
    print("Dataset deemed not usable due to lack of gene expression data.")
else:
    # Only proceed with normalization if we have gene data
    print("Normalizing gene symbols...")
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {gene_data_normalized.shape[0]} genes × {gene_data_normalized.shape[1]} samples")

    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_normalized.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
    
    # Extract clinical features from the original data source
    print("Extracting clinical features from the original source...")
    # Get background information and clinical data again
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    print("Extracted clinical features preview:")
    print(preview_df(selected_clinical_df))
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    
    # Save the extracted clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    print("Linking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data_normalized)
    print(f"Linked data shape: {linked_data.shape}")
    
    # Check if the linked data has adequate data
    if linked_data.shape[0] == 0 or linked_data.shape[1] <= 4:  # 4 is an arbitrary small number
        print("Error: Linked data has insufficient samples or features.")
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=True,
            df=linked_data,
            note="Failed to properly link gene expression data with clinical features."
        )
        print("Dataset deemed not usable due to linking failure.")
    else:
        # Handle missing values systematically
        print("Handling missing values...")
        linked_data_clean = handle_missing_values(linked_data, trait_col=trait)
        print(f"Data shape after handling missing values: {linked_data_clean.shape}")
        
        # Check if there are still samples after missing value handling
        if linked_data_clean.shape[0] == 0:
            print("Error: No samples remain after handling missing values.")
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=True,
                df=pd.DataFrame(),
                note="All samples were removed during missing value handling."
            )
            print("Dataset deemed not usable as all samples were filtered out.")
        else:
            # Check if the dataset is biased
            print("\nChecking for bias in feature variables:")
            is_biased, linked_data_final = judge_and_remove_biased_features(linked_data_clean, trait)
            
            # Conduct final quality validation
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=is_biased,
                df=linked_data_final,
                note="Dataset contains gene expression data for Crohn's Disease patients, examining response to Infliximab treatment."
            )
            
            # Save linked data if usable
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                linked_data_final.to_csv(out_data_file)
                print(f"Linked data saved to {out_data_file}")
                print(f"Final dataset shape: {linked_data_final.shape}")
            else:
                print("Dataset deemed not usable for trait association studies, linked data not saved.")

Normalizing gene symbols...
Gene data shape after normalization: 22464 genes × 35 samples


Normalized gene expression data saved to ../../output/preprocess/Psoriasis/gene_data/GSE183134.csv
Extracting clinical features from the original source...
Extracted clinical features preview:
{'GSM5551681': [0.0], 'GSM5551682': [0.0], 'GSM5551683': [0.0], 'GSM5551684': [0.0], 'GSM5551685': [0.0], 'GSM5551686': [0.0], 'GSM5551687': [0.0], 'GSM5551688': [0.0], 'GSM5551689': [0.0], 'GSM5551690': [0.0], 'GSM5551691': [0.0], 'GSM5551692': [0.0], 'GSM5551693': [0.0], 'GSM5551694': [1.0], 'GSM5551695': [1.0], 'GSM5551696': [1.0], 'GSM5551697': [1.0], 'GSM5551698': [1.0], 'GSM5551699': [1.0], 'GSM5551700': [1.0], 'GSM5551701': [1.0], 'GSM5551702': [1.0], 'GSM5551703': [1.0], 'GSM5551704': [1.0], 'GSM5551705': [1.0], 'GSM5551706': [1.0], 'GSM5551707': [1.0], 'GSM5551708': [1.0], 'GSM5551709': [1.0], 'GSM5551710': [1.0], 'GSM5551711': [1.0], 'GSM5551712': [1.0], 'GSM5551713': [1.0], 'GSM5551714': [1.0], 'GSM5551715': [1.0]}
Clinical data shape: (1, 35)
Clinical features saved to ../../output/pr

Data shape after handling missing values: (35, 22465)

Checking for bias in feature variables:
For the feature 'Psoriasis', the least common label is '0.0' with 13 occurrences. This represents 37.14% of the dataset.
The distribution of the feature 'Psoriasis' in this dataset is fine.



Linked data saved to ../../output/preprocess/Psoriasis/GSE183134.csv
Final dataset shape: (35, 22465)
